# Loading and Saving Data

to/from csv, hdf5, jld2, mat and xls files.

# Load Packages

The packages are loaded in the respective sections below. This allows you to run parts of this notebook without having to install all packages.

All files are written to and loaded from the subfolder "Results".

In [1]:
#using Compat, Missings        #in Julia 0.6 
using Dates                  #in Julia 0.7

include("printmat.jl")   #a function for prettier matrix printing

if !isdir("Results")
    error("create the subfolder Results before running this program")
end

# Saving and Loading a csv File

This functionality is built in to Julia. No package needed.

The simplest commands are
```
writedlm(FileName,matrix)
x = readdlm(FileName)
```

Extra arguments control the delimiters (for instance, ',') in the file, the type of data (Float, Int, etc), and whether the file starts with a header line with variable names.

For instance, if you want to specify that the delimter as comma and also disregard the first 3 lines (perhaps because the contain variables names etc), then use

```
x = readdlm(FileName,',',skipstart=3)
```

If you need more powerful write/read routines, try https://github.com/JuliaData/CSV.jl.

In [2]:
using DelimitedFiles  #in Julia 0.7        

A = copy(reshape(1:20,5,4))     #to be on the safe side: only try to save
                                #"independent" arrays, not reshapes or views 

writedlm("Results/NewCsvFile.csv",A,',')  #write csv file
println("NewCsvFile.csv has been created in the subfolder Results. Check it out.")

NewCsvFile.csv has been created in the subfolder Results. Check it out.


In [3]:
A2 = readdlm("Results/NewCsvFile.csv",',',Int)  #read csv file, try Float64 instead of Int

println("\nA (in memory):")
printmat(A)
println("\nA2 (read from csv file):")
printmat(A2)


A (in memory):
         1         6        11        16
         2         7        12        17
         3         8        13        18
         4         9        14        19
         5        10        15        20


A2 (read from csv file):
         1         6        11        16
         2         7        12        17
         3         8        13        18
         4         9        14        19
         5        10        15        20



# Saving and Loading an hdf5 File

hdf5 files are used in many computer languages. They can store different types of data: integers, floats, strings (but not Julia dates). 

The basic syntax is 
```
fh = h5open(FileName,"w")
    write(fh,"x",x,"y",y)
close(fh)

fh = h5open(FileName,"r")   #open for reading
    (x,y) = read(fh,"x","y")
close(fh)
```


The package HDF5 is at https://github.com/JuliaIO/HDF5.jl. 

See https://support.hdfgroup.org/products/java/hdfview/
for a program that lets you look at the contents of a hdf5 file. (It is not needed here.)

In [4]:
using HDF5   

A = copy(reshape(1:20,5,4))
B = 1
C = "Nice cat"

fh = h5open("Results/NewH5File.h5","w")    #open file for writing   
    write(fh,"A",A,"B",B,"C",C)
close(fh)                                  #close file

println("NewH5File.h5 has been created in the subfolder Results")

NewH5File.h5 has been created in the subfolder Results


In [5]:
fh = h5open("Results/NewH5File.h5","r")    #open for reading
     println("\nVariables in h5 file: ",names(fh))
    (A2,B2) = read(fh,"A","B")             #load some of the data
close(fh)

println("\nA from h5 file is")
printmat(A2)


Variables in h5 file: ["A", "B", "C"]

A from h5 file is
         1         6        11        16
         2         7        12        17
         3         8        13        18
         4         9        14        19
         5        10        15        20



# Saving and Loading a jld2 File

jld2 files can store very different types of data: integers, floats, strings, dictionaries, etc. It is a "dialect" of hdf5, designed to save different Julia objects (including Dates). 

The easiest syntax is 
```
save(FileName,"MatrixName1",matrix1,"MatrixName2",matrix2)
(x1,x2) = load(FileName,"MatrixName1","MatrixName2")  
```
(It also possible to use the same syntax as for HDF5, except that we use ```jldopen``` instead of ```h5open```.)

The package JLD2 is at https://github.com/simonster/JLD2.jl.

In [6]:
using FileIO, JLD2           #the FileIO package is alos needed        
                                        
A = copy(reshape(1:20,5,4))
B = 1
C = "Nice cat"
(A2,B2,C2) = (nothing,nothing,nothing)               #erase earlier results

save("Results/NewJldFile.jld2","A",A,"B",B,"C",C)       #write jld file

println("NewJldFile.jld2 has been created in the subfolder Results")

NewJldFile.jld2 has been created in the subfolder Results


In [7]:
x = load("Results/NewJldFile.jld2")                   #load entire file
println("The variables are: ",keys(x))               #list contents of the file 

(A2,B2) = load("Results/NewJldFile.jld2","A","B")     #read some of the data
println("\nA from jld2 file is")
printmat(A2)

The variables are: ["B", "A", "C"]

A from jld2 file is
         1         6        11        16
         2         7        12        17
         3         8        13        18
         4         9        14        19
         5        10        15        20



# (extra) Saving and Loading a Matlab mat File

The MAT package allows you to load/save (Matlab) mat files, which is another dialect of HDF5. 

See https://github.com/simonster/MAT.jl.

Warning: NOT YET WORKING ON JULIA 0.7/1.0

In [ ]:
using MAT     

A = copy(reshape(1:20,5,4))
B = 1
C = "Nice cat"
(A2,B2,C2) = (nothing,nothing,nothing)               #erase earlier results

fh = matopen("Results/NewMatFile.mat","w")   
    write(fh,"A",A)             #write one variable at a time                       
    write(fh,"B",B)
    write(fh,"C",C)
close(fh)

println("\nNewMatFile.mat has been created in the subfolder Results")

In [ ]:
fh = matopen("Results/NewMatFile.mat")           
    println("\nVariables in mat file: ",names(fh))
    (A2,B2) = read(fh,"A","B")                                
close(fh) 

println("\nA from mat file is ")
printmat(A2)

# (extra) Loading csv and Fixing Missing Values

The next cells show how to load an csv files with some missing values - and how to convert them to NaNs.

In [9]:
x = readdlm("Data/loadCsvTsT_Data.csv",',',skipstart=1)   #load the data

println("\nx")
printmat(x)


x
      1861    10.094     9.998          
      1862    10.128     9.994     0.037
      1863    10.156    10.044     0.092
      1864              10.061     0.098
      1865    10.197    10.076     0.078
      1866    10.234    10.064     0.061
      1867    10.242    10.060     0.025
      1868    10.186    10.057    -0.002
      1869    10.235    10.068     0.107
      1870    10.343    10.160     0.071
      1871    10.379    10.188     0.022
      1872    10.438    10.256    -0.022
      1873    10.494    10.315    -0.060
      1874    10.505    10.337     0.005
      1875    10.485    10.305     0.074



In [10]:
x2      = copy(x)
vv      = .!isa.(x,Number)               #locate cells that are not numbers,
x2[vv] .= NaN                            #then set them to NaN (or perhaps missing), and 
x2      = convert(Array{Float64},x2)     #convert the matrix from Any to Float64

println("\nafter fix")
printmat(x2)


after fix
  1861.000    10.094     9.998       NaN
  1862.000    10.128     9.994     0.037
  1863.000    10.156    10.044     0.092
  1864.000       NaN    10.061     0.098
  1865.000    10.197    10.076     0.078
  1866.000    10.234    10.064     0.061
  1867.000    10.242    10.060     0.025
  1868.000    10.186    10.057    -0.002
  1869.000    10.235    10.068     0.107
  1870.000    10.343    10.160     0.071
  1871.000    10.379    10.188     0.022
  1872.000    10.438    10.256    -0.022
  1873.000    10.494    10.315    -0.060
  1874.000    10.505    10.337     0.005
  1875.000    10.485    10.305     0.074



# (extra) Loading an xls File


You need python and python's xlrd libarary for this to work. There are two alternative commands (`readxl`  and `readxlsheet`), with slightly different functionality.

Missing values can be handled in the same way as for csv files (see above).

See https://github.com/davidanthoff/ExcelReaders.jl

In [11]:
using ExcelReaders

println("Approach 1: readxl")

data1 = readxl("Data/readXlsTsT_Data.xlsx","Data!B2:C11")       #reading a range
x1    = convert(Array{Float64},data1)            #convert from Any[] to Float64
println("\nNumeric part after conversion:")
printmat(x1)

Approach 1: readxl

Numeric part after conversion:
    16.660  -999.990
    16.850  -999.990
    16.930  -999.990
    16.980  -999.990
    17.080  -999.990
    17.030     7.000
    17.090     8.000
    16.760  -999.990
    16.670  -999.990
    16.720  -999.990



In [12]:
println("Approach 2: readxlsheet")

data2 = readxlsheet("Data/readXlsTsT_Data.xlsx","Data",skipstartrows=1)
x2    = convert(Array{Float64},data2[:,2:end])
println("\nNumeric part after conversion: ")
printmat(x2)

vv      = x2 .== -999.99
x2[vv] .= NaN                     #convert -999.99 to NaNs
println("\nNumeric part after changing -999.99 to NaN:")
printmat(x2)

Approach 2: readxlsheet

Numeric part after conversion: 
    16.660  -999.990
    16.850  -999.990
    16.930  -999.990
    16.980  -999.990
    17.080  -999.990
    17.030     7.000
    17.090     8.000
    16.760  -999.990
    16.670  -999.990
    16.720  -999.990


Numeric part after changing -999.99 to NaN:
    16.660       NaN
    16.850       NaN
    16.930       NaN
    16.980       NaN
    17.080       NaN
    17.030     7.000
    17.090     8.000
    16.760       NaN
    16.670       NaN
    16.720       NaN

